# Catégorisation de véhicules uniforme

On catégorise les véhicules en fonction de leur emission de CO2 réel.

In [2]:
import numpy as np
np.float=float
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
np.bool = bool
from sklearn.linear_model import Lasso

### 1. Travail d'Antoine sur les données (le meme que dans le fichier ML)

In [ ]:
data = pd.read_excel('data_BLP_caracteristics_CO2_corr.xlsx')
data2=pd.read_excel('ventes 2021_IPP.xlsx')
data.rename(columns={'type_cnit': 'Type'}, inplace=True) #renome le cnit
new_data = pd.merge(data, data2, on='Type', how='inner') #fusionne les 2 par le 


In [ ]:

datap=new_data.loc[new_data['N']>20] # on prend N ou bien volume ? 
datap = datap[['mean_cylindree','mean_co2_theorique','mean_ptr', 'mean_charge_utile','mean_puis_kw', 'mean_co2_reel', 'energ', 'mean_nb_decibels','Carrosserie', 'mean_ptac', 'VOLUME', 'N', 'Prix du véhicule']]

datap.dropna(inplace=True)

In [ ]:
datap.columns

Index(['mean_cylindree', 'mean_co2_theorique', 'mean_ptr', 'mean_charge_utile',
       'mean_puis_kw', 'mean_co2_reel', 'energ', 'mean_nb_decibels',
       'Carrosserie', 'mean_ptac', 'VOLUME', 'N', 'Prix du véhicule'],
      dtype='object')

In [ ]:
y=datap['mean_co2_reel']
dataOG=new_data['mean_co2_theorique']
dataEtat=datap['mean_co2_theorique']


for col in list(datap['energ'].unique()): 
    datap[col]=(datap['energ']==col)*1

for col in list(datap['Carrosserie'].unique()):
    datap[col]=(datap['Carrosserie']==col)*1

datap_train = datap.loc[-datap['energ'].isin(['EL'])] # sans EL
datap_train=datap_train.drop('energ', axis=1)
datap_train=datap_train.drop('Carrosserie', axis=1)
datap=datap.drop('energ', axis=1)
datap=datap.drop('Carrosserie', axis=1)
y_sel = datap_train['mean_co2_reel']
datap_train=datap_train.drop(['mean_co2_reel', 'N', 'VOLUME', 'Prix du véhicule'], axis=1)
datas=datap.drop(['mean_co2_reel', 'N', 'VOLUME', 'Prix du véhicule'], axis=1)

def normalization(dataToNormalize):
    columns=list(dataToNormalize.columns) # liste colonnes
    scaler=preprocessing.StandardScaler()
    for col in columns:
        dataToNormalize.loc[:,col]=scaler.fit_transform(dataToNormalize.loc[:,col].values.reshape(-1,1))

        
normalization(datap_train)
normalization(datas)

/tmp/ipykernel_344/1110656477.py:25: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dataToNormalize.loc[:,col]=scaler.fit_transform(dataToNormalize.loc[:,col].values.reshape(-1,1))


In [ ]:
datap_train.columns

Index(['mean_cylindree', 'mean_co2_theorique', 'mean_ptr', 'mean_charge_utile',
       'mean_puis_kw', 'mean_nb_decibels', 'mean_ptac', 'ES', 'GO', 'FE', 'EE',
       'EH', 'GH', 'EL', 'GN', 'GL', 'EG', 'FH', 'EN', 'BERLINE',
       'TS TERRAINS/CHEMINS', 'BREAK', 'MINIBUS', 'MONOSPACE COMPACT',
       'COMBISPACE', 'COUPE', 'MONOSPACE', 'MINISPACE', 'CABRIOLET'],
      dtype='object')

In [ ]:
datap.columns

Index(['mean_cylindree', 'mean_co2_theorique', 'mean_ptr', 'mean_charge_utile',
       'mean_puis_kw', 'mean_co2_reel', 'mean_nb_decibels', 'mean_ptac',
       'VOLUME', 'N', 'Prix du véhicule', 'ES', 'GO', 'FE', 'EE', 'EH', 'GH',
       'EL', 'GN', 'GL', 'EG', 'FH', 'EN', 'BERLINE', 'TS TERRAINS/CHEMINS',
       'BREAK', 'MINIBUS', 'MONOSPACE COMPACT', 'COMBISPACE', 'COUPE',
       'MONOSPACE', 'MINISPACE', 'CABRIOLET'],
      dtype='object')

In [ ]:
datap

,mean_cylindree,mean_co2_theorique,mean_ptr,mean_charge_utile,mean_puis_kw,mean_co2_reel,mean_nb_decibels,mean_ptac,VOLUME,N,...,BERLINE,TS TERRAINS/CHEMINS,BREAK,MINIBUS,MONOSPACE COMPACT,COMBISPACE,COUPE,MONOSPACE,MINISPACE,CABRIOLET
0,4806.000000,286.000000,4700.0,530.000000,368.00000,358.498030,77.0,2500.0,1,203,...,1,0,0,0,0,0,0,0,0,0
1,1598.000000,147.000000,2895.0,455.628490,86.00000,181.021890,72.0,1645.0,1,6318,...,0,1,0,0,0,0,0,0,0,0
3,1582.000000,108.000000,3420.0,560.456938,81.00000,151.611933,77.0,1920.0,1,419,...,1,0,0,0,0,0,0,0,0,0
7,1560.000000,100.000000,3160.0,569.984985,88.00000,140.958233,72.0,1860.0,1,3996,...,1,0,0,0,0,0,0,0,0,0
9,1198.981607,104.000000,2366.0,570.998824,59.99908,149.158444,76.0,1546.0,1,117339,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54157,998.000000,127.086957,2830.0,530.000000,88.30000,152.752174,72.0,1830.0,18,23,...,1,0,0,0,0,0,0,0,0,0
54158,998.000000,127.086957,2830.0,530.000000,88.30000,152.752174,72.0,1830.0,1,23,...,1,0,0,0,0,0,0,0,0,0
54202,999.000000,126.750000,2690.0,499.750000,91.90000,139.100000,72.0,1690.0,5,28,...,1,0,0,0,0,0,0,0,0,0
54203,999.000000,126.750000,2690.0,499.750000,91.90000,139.100000,72.0,1690.0,21,28,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
y_sel

0        358.498030
1        181.021890
3        151.611933
7        140.958233
9        149.158444
            ...    
54157    152.752174
54158    152.752174
54202    139.100000
54203    139.100000
54204    139.100000
Name: mean_co2_reel, Length: 39536, dtype: float64

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(datap_train, y_sel, test_size=0.2)
lasso_model = Lasso(alpha=0.2)
lasso_model.fit(x_train, y_train)


Lasso(alpha=0.2)

In [ ]:
test_predictions_lasso = lasso_model.predict(x_test)

print(f"RMSE_model = {round(sqrt(mean_squared_error(y_test,test_predictions_lasso)),3)}")

print(f"R2_score_model = {round(r2_score(y_test, test_predictions_lasso),3)}")

RMSE_model = 9.302
R2_score_model = 0.896


In [ ]:
test_predictions_lasso

array([154.81799419, 170.37548335, 180.88261451, ..., 213.71257983,
       116.92435795, 168.75329386])

## 2. On catégorise les véhicules selon leur emission reelle de co2

In [ ]:
x_test.columns

Index(['mean_cylindree', 'mean_co2_theorique', 'mean_ptr', 'mean_charge_utile',
       'mean_puis_kw', 'mean_nb_decibels', 'mean_ptac', 'ES', 'GO', 'FE', 'EE',
       'EH', 'GH', 'EL', 'GN', 'GL', 'EG', 'FH', 'EN', 'BERLINE',
       'TS TERRAINS/CHEMINS', 'BREAK', 'MINIBUS', 'MONOSPACE COMPACT',
       'COMBISPACE', 'COUPE', 'MONOSPACE', 'MINISPACE', 'CABRIOLET'],
      dtype='object')

In [ ]:
datas.columns

Index(['mean_cylindree', 'mean_co2_theorique', 'mean_ptr', 'mean_charge_utile',
       'mean_puis_kw', 'mean_nb_decibels', 'mean_ptac', 'ES', 'GO', 'FE', 'EE',
       'EH', 'GH', 'EL', 'GN', 'GL', 'EG', 'FH', 'EN', 'BERLINE',
       'TS TERRAINS/CHEMINS', 'BREAK', 'MINIBUS', 'MONOSPACE COMPACT',
       'COMBISPACE', 'COUPE', 'MONOSPACE', 'MINISPACE', 'CABRIOLET'],
      dtype='object')

In [ ]:
lasso_model.predict(x_test)

array([154.81799419, 170.37548335, 180.88261451, ..., 213.71257983,
       116.92435795, 168.75329386])

In [ ]:
# les données
datap['CO2reel'] = lasso_model.predict(datas)

datap.loc[datap['EL']==1, 'CO2reel']=0

CO2reel=datap['CO2reel']

#Choix du nombre de catégories
N = 30
nb_vehicules = datap['VOLUME'].sum()

In [ ]:
print(datap)

       mean_cylindree  mean_co2_theorique  mean_ptr  mean_charge_utile  \
0         4806.000000          286.000000    4700.0         530.000000   
1         1598.000000          147.000000    2895.0         455.628490   
3         1582.000000          108.000000    3420.0         560.456938   
7         1560.000000          100.000000    3160.0         569.984985   
9         1198.981607          104.000000    2366.0         570.998824   
...               ...                 ...       ...                ...   
54157      998.000000          127.086957    2830.0         530.000000   
54158      998.000000          127.086957    2830.0         530.000000   
54202      999.000000          126.750000    2690.0         499.750000   
54203      999.000000          126.750000    2690.0         499.750000   
54204      999.000000          126.750000    2690.0         499.750000   

       mean_puis_kw  mean_co2_reel  mean_nb_decibels  mean_ptac  VOLUME  \
0         368.00000     358.498030  

On categorise de maniere uniforme selon le co2 reel

In [ ]:
# 1. On trie les modeles par CO2 croissant
datap = datap.sort_values(by='CO2reel')

#nombre de catégories
N = 60
min_value = datap[datap['CO2reel'] > 0]['CO2reel'].min()
maxCO2 = datap['CO2reel'].max()
bins_limits = np.concatenate((np.array([0]), np.linspace(min_value, maxCO2, N-2), np.array([maxCO2+1])))



# 2. On decoupe en tranche de CO2 
datap['CO2bins'] = pd.cut(datap['CO2reel'], bins=bins_limits, include_lowest=True, right=False)


In [ ]:
bins_limits

array([  0.        ,  66.38155961,  69.77837436,  73.1751891 ,
        76.57200384,  79.96881859,  83.36563333,  86.76244808,
        90.15926282,  93.55607756,  96.95289231, 100.34970705,
       103.74652179, 107.14333654, 110.54015128, 113.93696602,
       117.33378077, 120.73059551, 124.12741026, 127.524225  ,
       130.92103974, 134.31785449, 137.71466923, 141.11148397,
       144.50829872, 147.90511346, 151.3019282 , 154.69874295,
       158.09555769, 161.49237244, 164.88918718, 168.28600192,
       171.68281667, 175.07963141, 178.47644615, 181.8732609 ,
       185.27007564, 188.66689038, 192.06370513, 195.46051987,
       198.85733461, 202.25414936, 205.6509641 , 209.04777885,
       212.44459359, 215.84140833, 219.23822308, 222.63503782,
       226.03185256, 229.42866731, 232.82548205, 236.22229679,
       239.61911154, 243.01592628, 246.41274103, 249.80955577,
       253.20637051, 256.60318526, 260.        , 417.66824141])

In [ ]:
print(datap['CO2reel'].max())

416.668241412398


In [ ]:
datap['CO2bins']

286         [0.0, 66.382)
3326        [0.0, 66.382)
288         [0.0, 66.382)
3325        [0.0, 66.382)
287         [0.0, 66.382)
               ...       
33882    [260.0, 417.668)
33883    [260.0, 417.668)
33884    [260.0, 417.668)
47997    [260.0, 417.668)
47996    [260.0, 417.668)
Name: CO2bins, Length: 39549, dtype: category
Categories (59, interval[float64, left]): [[0.0, 66.382) < [66.382, 69.778) < [69.778, 73.175) < [73.175, 76.572) ... [249.81, 253.206) < [253.206, 256.603) < [256.603, 260.0) < [260.0, 417.668)]

On créé les vecteurs prix et CO2reel necessaires a l'optimisation. 

In [ ]:

# Group by the new 'Co2_prf_bins' column and count the number of vehicles in each bin
price_by_bin = datap.groupby('CO2bins')['Prix du véhicule'].mean()
p=price_by_bin

# Get the mean CO2 from each bin
meanCO2_by_bin = datap.groupby('CO2bins')['CO2reel'].mean()
CO2reel=meanCO2_by_bin

CO2theorique=datap.groupby('CO2bins')['mean_co2_theorique'].mean()

In [ ]:
datap.groupby('CO2bins')['VOLUME'].sum()

NameError: name 'datap' is not defined

In [ ]:
CO2reel

CO2bins
[0.0, 66.382)           0.000000
[66.382, 69.778)       67.577801
[69.778, 73.175)       72.195569
[73.175, 76.572)       75.287266
[76.572, 79.969)       78.158437
[79.969, 83.366)       81.458990
[83.366, 86.762)       85.266236
[86.762, 90.159)       88.514416
[90.159, 93.556)       91.468617
[93.556, 96.953)       95.459179
[96.953, 100.35)       98.485057
[100.35, 103.747)     101.014831
[103.747, 107.143)    105.616654
[107.143, 110.54)     108.622090
[110.54, 113.937)     112.298841
[113.937, 117.334)    116.285579
[117.334, 120.731)    118.888034
[120.731, 124.127)    122.365441
[124.127, 127.524)    125.818717
[127.524, 130.921)    129.145126
[130.921, 134.318)    132.872066
[134.318, 137.715)    136.086716
[137.715, 141.111)    139.439085
[141.111, 144.508)    142.748089
[144.508, 147.905)    146.370089
[147.905, 151.302)    149.586832
[151.302, 154.699)    152.898848
[154.699, 158.096)    156.423323
[158.096, 161.492)    159.780249
[161.492, 164.889)    163.132894
[1

In [ ]:
CO2theorique

CO2bins
[0.0, 66.382)           0.000000
[66.382, 69.778)       29.509876
[69.778, 73.175)       30.162466
[73.175, 76.572)       24.947037
[76.572, 79.969)       28.200124
[79.969, 83.366)       30.722673
[83.366, 86.762)       33.244016
[86.762, 90.159)       34.544430
[90.159, 93.556)       35.478795
[93.556, 96.953)       36.812528
[96.953, 100.35)       43.891655
[100.35, 103.747)      45.827230
[103.747, 107.143)     39.464457
[107.143, 110.54)      45.471847
[110.54, 113.937)      57.847180
[113.937, 117.334)     90.094205
[117.334, 120.731)     83.984200
[120.731, 124.127)    103.077162
[124.127, 127.524)    104.023397
[127.524, 130.921)    110.642539
[130.921, 134.318)    114.623954
[134.318, 137.715)    118.500051
[137.715, 141.111)    118.144946
[141.111, 144.508)    122.301595
[144.508, 147.905)    125.939817
[147.905, 151.302)    127.081873
[151.302, 154.699)    133.100147
[154.699, 158.096)    135.463569
[158.096, 161.492)    138.334133
[161.492, 164.889)    141.956825
[1